# Instalowanie zależności

In [1]:
!pip install langchain cassio datasets huggingface-hub langchain-community PyPDF2 gcsfs==2024.9.0.post1 openai==1.55.3 tiktoken python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existin

# Import bibliotek

In [2]:
# Biblioteki Langchain do wykorzystania
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

# Wsparcie do przeszukiwania zbioru danych za pomocą Hugging Face
from datasets import load_dataset

# Za pomocą CassIO, silnika zasilającego integracje AstraDB z Langchain
# Również zaincjujemy połączenie z bazą danych
import cassio
# Biblioteka do obsługi Plików PDF
from PyPDF2 import PdfReader

from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage

from langchain.text_splitter import CharacterTextSplitter

# Setup Bazy Danych oraz API

In [7]:
from google.colab import files
import os
# Prześlij plik .env
uploaded = files.upload()

# Użyj dotenv do załadowania zmiennych
from dotenv import load_dotenv
load_dotenv()

ASTRA_DB_APPLICATION_TOKEN = os.getenv('ASTRA_DB_APPLICATION_TOKEN')
ASTRA_DB_ID = os.getenv('ASTRA_DB_ID')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

Saving .env to .env (1)


# Załadowanie Pliku PDF

In [8]:
# Ścieżka do odczytu pliku PDF
pdfreader = PdfReader('Grammar.pdf')

# Odczyt Tekstu Z Pliku PDF

In [9]:
# Odczyt textu z pliku pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
  content = page.extract_text()
  if content:
    raw_text += content

# Inicjalizacja Połączenia Z Bazą Danych

In [10]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

# Stworzenie embeddingu Langchain oraz obiektów LLM do późniejszego wykorzystania

In [16]:
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0.7, model='gpt-4o-mini')
embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
print()
print(f"Configured model: {llm.model_name}")


Configured model: gpt-4o-mini


# Utworzenie Langchain Vector Store (Bazy danych Wektorowej) wspieranej przez AstraDB

In [12]:
astra_vector_store = Cassandra(
    embedding=embedding,
    table_name="qa_mini_demo",
    session=None,
    keyspace=None,
)

# Rozdzielenie tekstu

In [13]:
# Musimy rozdzielić tekst korzystając z Character Text Splitter ponieważ nie powinno to zwiększyć wielkości tokena
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

# Ładowane zbioru danych do Vector Store

In [14]:
astra_vector_store.add_texts(texts[:200])

print("Inserted %i headlines." % len(texts[:200]))

astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 200 headlines.


# Uruchamiamy Cykl Q&A (Pytanie i odpowiedź)

In [15]:
first_question = True

while True:
  if first_question:
    query_text = input("\nEnter your question (or type 'quit' to exit): ").strip()
  else:
    query_text = input("\nWhat's your next question (or type 'quit' to exit): ").strip()

  if query_text.lower() == 'quit':
    break

  if query_text == "":
    continue

  first_question = False

  print(f"\nQUESTION: \"{query_text}\"")

  relevant_docs = astra_vector_store.similarity_search_with_score(query_text, k=4)

  if relevant_docs:
      print("RELEVANT DOCUMENTS:")
      combined_context = "\n".join([doc.page_content[:400] for doc, score in relevant_docs])
      for doc, score in astra_vector_store.similarity_search_with_score(query_text, k=4):
        print("   [%0.4f] \"%s ...\"" % (score, doc.page_content[:84]))
      # Konstruowanie zapytania z wiadomościami: SYSTEM i UŻYTKOWNIK
      messages = [
          SystemMessage(content="You are a helpful assistant."),
          HumanMessage(content=f"Context: {combined_context}\n\nQuestion: {query_text}")
      ]

      # Wykonanie zapytania do modelu
      response = llm(messages)

      # Odpowiedź z modelu to obiekt AIMessage, uzyskujemy zawartość za pomocą atrybutu 'content'
      answer = response.content.strip()

      print(f"\nANSWER: \"{answer}\"")
  else:
      print("No relevant documents found.")


Enter your question (or type 'quit' to exit): Czego nauczy mnie ta książka?

QUESTION: "Czego nauczy mnie ta książka?"


RELEVANT DOCUMENTS:


<ipython-input-15-f3ec9d8df65c>:33: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(messages)


   [0.9103] "że John niesamowicie sprawnie  i szybko si ę uczył. Wszędzie przy-
klejał żółte kart ..."
   [0.8965] "ilość czasu nie jest wykuta w kamieniu; po prostu niech b ędzie to 
taki okres, któr ..."
   [0.8765] "tion for further study in English grammar and usage. It will benefit students who ar ..."
   [0.8765] "tion for further study in English grammar and usage. It will benefit students who ar ..."

ANSWER: "Ta książka nauczy Cię podstaw gramatyki i użycia języka angielskiego. Będzie przydatna zarówno dla osób, które uczą się tych zagadnień po raz pierwszy, jak i dla tych, którzy chcą zrecenzować wcześniej poznane koncepcje. Zrozumienie, jak działa język, pozwoli Ci skuteczniej go używać, co może poprawić Twoją komunikację osobistą, wyniki w szkole oraz karierę zawodową. Książka ma na celu rozwijanie umiejętności językowych, co przyniesie korzyści w różnych aspektach życia."

What's your next question (or type 'quit' to exit): quit
